# **Práctica 4.2: Redes neuronales (Clasificación)**

<hr>

## **1. Introducción**
En la práctica anterior aprendimos a resolver problemas de regresión con redes neuronales. En esta práctica, exploraremos la resolución de **problemas de clasificación**.

Hasta ahora, nos hemos centrado en problemas de clasificación binaria pero en esta práctica abordaremos también dos nuevos tipos.

### **Objetivos**
En esta práctica aprenderás a:
* Distinguir entre tipos de problemas de clasificación.
* Modificar una red neuronal para aprender problemas de clasificación.
* Transformar variables categoricas en numéricas.

Comenzamos cargando una vez más nuestros datos:

In [ ]:
import pandas as pd

seed = 2533
data = pd.read_pickle("https://raw.githubusercontent.com/AIC-Uniovi/Sistemas-Inteligentes/refs/heads/main/datasets/f1_23_monaco.pkl")

<hr>

## **2. Problemas de clasificación binaria**

Vamos a intentar resolver un problema similar al de la práctica 3 de clasificación, es decir: 

<div class="alert alert-block alert-success">
    <b>Crear un modelo que, dado el tiempo (en segundos) de los dos primeros sectores de un piloto de <i>Aston Martin</i> (<code>"Sector1Time", "Sector2Time"</code>), se prediga si ese tiempo lo realizó <i>Alonso</i> o no (<i>Stroll</i>).</b>
</div>

Como siempre, lo primero será crear los datasets necesarios para entrenar un modelo.

### **2.1. Preprocesado de datos**

Creamos la variable <code>data_aston</code> con las filas y columnas necesarias para entrenar nuestros modelos.


In [ ]:
data_aston = data.loc[data.Team=="Aston Martin"][["Sector1Time", "Sector2Time", "Driver"]].copy()
data_aston["Sector1Time"] = data_aston["Sector1Time"].dt.total_seconds()
data_aston["Sector2Time"] = data_aston["Sector2Time"].dt.total_seconds()

<div class="alert alert-block alert-info">
    <b>Ejercicio:</b> Crea la columna <code>Class</code> dentro del DataFrame <code>data_aston</code> para que valga cero siempre que el piloto no sea Alonso y 1 en el caso contrario. 
</div>

In [ ]:
# Tu código aquí

<div class="alert alert-block alert-info">
    <b>Ejercicio:</b> Separa las X e Y del dataframe <code>data_aston</code> , divide en entrenamiento y test (80/20) fijando la semilla y finalmente <b>estandariza</b> las X.
</div>

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Tu código aquí

### **2.2. Aprendizaje automático**

Con los datos listos, entrenaremos y evaluaremos de nuevo los modelos de aprendizaje automático ya conocidos para poder compararlos con nuestro nuevo sistema.

<div class="alert alert-block alert-info">
    <b>Ejercicio:</b> Entrena y evalúa los modelos restantes (<i>Regresión Logística</i>, <i>K-Nearest Neighbors</i>, <i>Árboles de decisión</i> y <i>SVC</i>) utilizando la siguiente función.
</div>

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
from sklearn.dummy import DummyClassifier

def evaluate_model(Y_test, preds_test, model_name, average="binary"):
    metrics = {
        "Métrica": ["Accuracy", "F1"],
        "TEST": [accuracy_score(Y_test, preds_test), f1_score(Y_test, preds_test, average=average)]
    }
    df = pd.DataFrame(metrics)
    print(f"Resultados para {model_name}:")
    print(df.to_string(index=False))
    print()
    
# Baseline Random
baseline_random = DummyClassifier(strategy="uniform")
baseline_random.fit(X_train, Y_train)
preds_test = baseline_random.predict(X_test)
evaluate_model(Y_test, preds_test, "Baseline Random")

# Baseline Zero-R
baseline_zero = DummyClassifier(strategy="most_frequent")
baseline_zero.fit(X_train, Y_train)
preds_test = baseline_zero.predict(X_test)
evaluate_model(Y_test, preds_test, "Baseline Zero-R")

# Tu código aquí

Los resultados han de ser algo así:

<center>

| Modelo                | Accuracy (test) | F1 (test) |
|-----------------------|-----------------|-----------|
| Baseline Random       | 0.522           | 0.560     |
| Baseline Zero-R       | 0.565           | 0.722     |
| Regresión Logística   | 0.565           | 0.722     |
| KNN                   | 0.957           | 0.963     |
| Árboles de Decisión   | 0.826           | 0.867     |
| SVC                   | 0.913           | 0.929     |

</center>

##### **Visualizar datos y modelos**

En este caso, nuestro problema tiene dos entradas y una salida (la clase). Como ya vimos en la parte de regresión, trabajar con tan pocas dimensiones nos permite visualizar el comportamiento de los datos y de los modelos que estamos aprendiendo.

Gracias a esta posibilidad, podemos analizar de antemano si la relación entre las entradas y salidas puede resolverse con modelos lineales o, por el contrario, requiere de un enfoque no lineal.

A continuación te proporcionamos la función que visualiza los datos y, dado un modelo, realiza una serie de predicciones para dibujar su **frontera de decisión**:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Función para visualizar los datos y la frontera de decisión del modelo
def plot_decision_boundary(X_train, Y_train, X_test, Y_test, model, model_name):
    plt.figure(figsize=(8, 6))

    # Crear una malla de puntos en el rango de los datos de Train y Test
    x_min, x_max = min(X_train[:, 0].min(), X_test[:, 0].min()) - 0.5, max(X_train[:, 0].max(), X_test[:, 0].max()) + 0.5
    y_min, y_max = min(X_train[:, 1].min(), X_test[:, 1].min()) - 0.5, max(X_train[:, 1].max(), X_test[:, 1].max()) + 0.5
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100), np.linspace(y_min, y_max, 100))

    # Predecir la probabilidad para cada punto de la malla
    grid = np.c_[xx.ravel(), yy.ravel()]
    Z = model.predict(grid).reshape(xx.shape)

    # Dibujar la frontera de decisión
    contour = plt.contourf(xx, yy, Z, levels=[0, 0.5, 1], alpha=0.7, cmap="coolwarm")

    # Añadir colorbar
    plt.colorbar(contour)

    # Visualizar los puntos de Train
    plt.scatter(X_train[:, 0], X_train[:, 1], c=Y_train, cmap="coolwarm", edgecolors="k", label="Train Data")
    
    # Visualizar los puntos de Test
    plt.scatter(X_test[:, 0], X_test[:, 1], c=Y_test, cmap="coolwarm", marker="X", label="Test Data")

    # Etiquetas y leyenda
    plt.xlabel("Sector1Time")
    plt.ylabel("Sector1Time")
    plt.title(f"Frontera de decisión: {model_name}")
    plt.legend()
    plt.show()

plot_decision_boundary(X_train, Y_train, X_test, Y_test, baseline_random, "Baseline Aleatorio")
plot_decision_boundary(X_train, Y_train, X_test, Y_test, baseline_zero, "Baseline Zero-R")

<div class="alert alert-block alert-info">
    <b>Ejercicio:</b> Dibuaja la frontera de decisión para el resto de modelos y trata de entender los resultados.
</div>

In [ ]:
    # Tu código aquí

<div class="alert alert-block alert-info">
    <b>Ejercicio:</b> ¿Crees que el problema es lineal o no lineal? Analizando las fronteras de decisión, ¿qué modelos son no lineales?
</div>

Tu respuesta aquí

### **2.3. Red neuronal**

Crearemos ahora una red neuronal desde cero para resolver este problema. Recuerda que los pasos son los siguientes:

1) Crear la arquitectura del modelo.
2) Detallar el optimizador, la función de pérdida y compilar.
3) Entrenar y evaluar.

Vamos a fijar las semillas y crear la función para dibujar la evolución del entrenamiento del modelo: 

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
import pandas as pd
import numpy as np
import os, random

# Fijar las semillas de las librerías para que los resultados se repitan.
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

def plot_loss_history(history):
    # Extraer datos del historial
    loss = history.history['loss']
    val_loss = history.history.get('val_loss', []) # Usamos lista vacía por defecto
    epochs = range(1, len(loss) + 1)
    # Construir el DataFrame
    data = pd.DataFrame({ 'Epoch': epochs, 'Loss': loss, 'Type': 'Train' })
    # Si existe validación, concatenamos esos datos
    if val_loss:
        val_df = pd.DataFrame({ 'Epoch': epochs, 'Loss': val_loss, 'Type': 'Validation' })
        data = pd.concat([data, val_df], ignore_index=True)
    # Crear el gráfico
    plt.figure(figsize=(10, 5))
    sns.lineplot(data=data, x="Epoch", y="Loss", hue="Type")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title("Evolución de la Pérdida durante el Entrenamiento")
    plt.grid(True, linestyle='--', alpha=0.7)
    # Solo mostrar leyenda si hay más de un tipo de dato
    if val_loss: plt.legend(title="Conjunto")
    else: plt.legend().remove() # Evita leyendas vacías o innecesarias
    plt.show()

##### **Activación en la última capa y loss**

Como se mencionó en la práctica anterior, al crear una red neuronal es crucial seleccionar adecuadamente tanto la **función de activación de la última capa** como la **función de pérdida**.

En un problema de regresión, la última capa generalmente no utiliza una función de activación para evitar que las predicciones se limiten a un rango específico. Sin embargo, si los valores que se intentan predecir son siempre positivos, se podría aplicar una función *ReLU*.

<center>
    <div style="border-radius:5px; padding:10px; background:white; max-width:900px">
        <img src="https://i.imgur.com/e7kd5fs.png">   
    </div>
</center>

Nos enfrentamos ahora a un problema de **clasificación binaria**, donde buscamos predecir una probabilidad, es decir, un valor comprendido entre $0$ y $1$. Por consiguiente, debemos emplear una función **sigmoide** como función de activación en la capa final.

<div class="alert alert-block alert-warning">
    <strong>Nota:</strong> Recuerda que una red neuronal que <b>solo</b> tiene una función de activación en la última capa <b>no puede aprender problemas no lineales</b>; para eso se requieren funciones de activación en las capas ocultas.
</div>

Otro elemento que tenemos que cambiar respecto de los problemas de regresión es la **función de pérdida** o loss. 

<div class="alert alert-block alert-warning">
    <strong>Nota:</strong> En un problema de clasificación binaria, no se deben usar funciones de pérdida diseñadas para regresión, como el <b>error absoluto medio</b> (MAE), ya que están orientadas a problemas donde las salidas <b>son valores continuos y no probabilidades</b>.
</div>

Por tanto, además de añadir una sigmoide a la capa de salida, tendremos también que cambiar la función de pérdida a **Binary Crossentropy**. Lo que nos deja con la siguiente tabla: 

<center>

| Tipo de problema              | Función de activación en la última capa         | Función de pérdida    | En *keras*                                  |
|-------------------------------|-------------------------------------------------|-----------------------|---------------------------------------------|
| *Regresión*                   | Ninguna o *ReLU* (si los valores son positivos) | *MAE* o *MSE*         | `mean_average_error` o `mean_squared_error` |
| *Clasificación Binaria*       | *Sigmoide*                                      | *Binary Crossentropy* | `binary_crossentropy`                       |

</center>



<div class="alert alert-block alert-info">
    <b>Ejercicio:</b> Crea, dentro de la función proporcionada, una red neuronal de <b>clasificación binaria</b> con una sola capa. Entrena, dibuja la evolución de la loss utilizando la función <code>plot_loss_history</code> y analiza su frontera de decisión.
    <hr>
    Entrena con un conjunto de validación del 20%, durante 300 epochs, con un tamaño de batch de 16 y un learning rate de 0,005.
</div>

In [ ]:
def red_neuronal_uno(learning_rate):
    # Creamos y compilamos el modelo
    
    # Tu código aquí

    return model

# Creamos la red desde cero
model_1 = red_neuronal_uno(learning_rate = 0.005)

# Entrenamos
# Tu código aquí

# Visualizar entrenamiento
# Tu código aquí

# Frontera de decisión
# Tu código aquí

<div class="alert alert-block alert-info">
    <b>Ejercicio:</b> Evalúa el modelo anterior en Test utilizando <code>.predict()</code> y <code>evaluate_model</code>. Añade el resultado a la tabla.
    <hr>
    En este caso <b>no</b> vamos a intentar buscar los mejores hiperparámetros; como has visto, este modelo es lineal y no va a ser capaz de resolver nuestro problema no lineal.
</div>

<div class="alert alert-block alert-warning">
    <strong>Nota:</strong> Las redes neuronales no devuelven valores binarios (0 o 1) de forma nativa, sino probabilidades (valores decimales). Las métricas no permiten decimales.
    <hr>
    Para obtener las métricas deberás aplicar un umbral de decisión. Comúnmente se utiliza 0.5 para convertir los valores superiores a este en 1 y los inferiores en 0.
</div>

<center>

| Modelo                | Accuracy (test) | F1 (test) |
|-----------------------|-----------------|-----------|
| Baseline Random       | 0.522           | 0.560     |
| Baseline Zero-R       | 0.565           | 0.722     |
| Regresión Logística   | 0.565           | 0.722     |
| KNN                   | 0.957           | 0.963     |
| Árboles de Decisión   | 0.826           | 0.867     |
| SVC                   | 0.913           | 0.929     |
| Red Neuronal Lineal   |                 |           |

</center>

In [ ]:
# Evaluamos
# Tu código aquí

<div class="alert alert-block alert-info">
    <b>Ejercicio:</b> Crea una red neuronal <u>no lineal</u> de clasificación binaria y busca el mejor learning rate. Entrena el modelo final con el mejor hiperparámetro y evalúa en test. Rellena ambas tablas.
    <hr>
    Fija el conjunto de validación al 20%, las epocas a 500 y el batch a 16. Para entrenar el modelo final no es necesario el conjunto de validación.
    <hr style="margin-bottom:5px">
    Visualiza también la frontera de decisión verificando que el modelo aprendido es no lineal.
</div>


<center>

| Modelo                     | Loss (train)  | Loss (val) |
|----------------------------|---------------|------------|
| *Red Neuronal (lr=0.001)*  |               |            |
| *Red Neuronal (lr=0.005)*  |               |            |
| *Red Neuronal (lr=0.01)*   |               |            |

</center>

<br>

<center>

| Modelo                 | Accuracy (test) | F1 (test) |
|------------------------|-----------------|-----------|
| Baseline Random        | 0.522           | 0.560     |
| Baseline Zero-R        | 0.565           | 0.722     |
| Regresión Logística    | 0.565           | 0.722     |
| KNN                    | 0.957           | 0.963     |
| Árboles de Decisión    | 0.826           | 0.867     |
| SVC                    | 0.913           | 0.929     |
| Red Neuronal Lineal    |                 |           |
| Red Neuronal No Lineal |                 |           |

</center>

In [ ]:
def red_neuronal_dos(learning_rate):
    # Creamos y compilamos el modelo
    
    # Tu código aquí

    return model

# Creamos la red desde cero
model_2 = red_neuronal_dos(learning_rate = 0.001)

# Entrenamos
# Tu código aquí

# Visualizar entrenamiento
# Tu código aquí

# Repetir para otro valor del hiperparámetro

In [ ]:
# Entrenar modelo final (sin validación)
# Tu código aquí

# Evaluar en test
# Tu código aquí

# Frontera de decisión
# Tu código aquí

<div class="alert alert-block alert-info">
    <b>Ejercicio:</b> Entra en la siguiente web para visualizar en entrenamiento y frontera de decisión de una red neuronal para diferentes problemas de clasificación binaria: <a href="https://playground.tensorflow.org/">https://playground.tensorflow.org/</a>
</div>

<hr>

## **3. Problemas de multiclasificación**

Hasta ahora, nuestros problemas de clasificación siempre se han centrado en la clasificación binaria, pero como sabes, existen más tipos de problemas de este tipo.

Vamos a intentar resolver ahora un problema **multiclase**, es decir, un problema donde cada ejemplo puede pertenecer **a una entre varias clases posibles**.

<div class="alert alert-block alert-success">
    <b>Crear un modelo que, dado el tiempo (en segundos) de los sectores (<code>"Sector1Time", "Sector2Time" y "Sector3Time"</code>), las velocidades (<code>"SpeedI1", "SpeedI2", "SpeedFL" y "SpeedST"</code>) y los datos del neumático (<code>"Compound" y "TyreLife"</code>) sea capaz de predecir el <i>equipo</i> (<code>"Team"</code>) del coche que realizó dicha vuelta.</b> 
</div>
  
Como siempre, lo primero será crear los datasets necesarios para entrenar un modelo.

### **3.1. Preprocesado de datos**

Creamos la variable <code>data_teams</code> con las filas y columnas necesarias para entrenar nuestros modelos.

In [ ]:
relevant_cols = ["Sector1Time", "Sector2Time", "Sector3Time", "SpeedI1", "SpeedI2", "SpeedFL", "SpeedST", "Compound", "TyreLife", "Team"]
data_teams = data[relevant_cols].copy()
data_teams = data_teams.dropna().reset_index(drop=True) # Eliminamos las filas con algún valor nulo

data_teams["Sector1Time"] = data_teams["Sector1Time"].dt.total_seconds()
data_teams["Sector2Time"] = data_teams["Sector2Time"].dt.total_seconds()
data_teams["Sector3Time"] = data_teams["Sector3Time"].dt.total_seconds()

<div class="alert alert-block alert-info">
    <b>Ejercicio:</b> Crea un <code>countplot()</code> de la columna <code>"Team"</code> del DataFrame <code>data_teams</code> para ver si existe desbalanceo de clases. ¿Crees que lo hay?
</div>

In [ ]:
plt.figure(figsize=(15,6))

# Tu código aquí

plt.show()

##### **Codificación: De categorías a números**

Los modelos solo entienden de números, pero la forma en que debemos preparar los datos varía según si estamos configurando las **entradas** o la **salida**, y según la **librería** que utilicemos.

<div class="alert alert-block alert-warning">
    <strong>Nota:</strong> Las entradas (X) siempre deben ser numéricas para todos los modelos y librerías.
    <br>
    Las salidas (y) solo es obligatorio codificarlas para Redes Neuronales.
</div>

**Librerías**

Tanto *Scikit-Learn* como *Keras* **siempre** necesitan que las características de **entrada** sean números. En las **salidas** es donde encontramos la principal diferencia:

* **Scikit-Learn:** Nos permite pasarle la variable objetivo `Team` directamente como **texto**. El modelo se encarga de mapear estas etiquetas automáticamente durante el entrenamiento.
* **Keras:** Las redes neuronales no tienen esa gestión automática. Para que la capa de salida funcione (especialmente con *Softmax*), debemos transformar las etiquetas de texto en **vectores numéricos (One-Hot)** manualmente antes de entrenar.

**Codificación (entrada o salida)**

* **Variables Binarias:** (ej: 'Sí'/'No' o 'Alonso'/'Stroll') las convertimos en $0$ y $1$.
* **Variables con varias categorías:** (ej: la variable `Compound`). No podemos usar números correlativos (1, 2, 3...) porque el modelo pensaría que una categoría es "mayor" que otra. Usamos **One-Hot Encoding** para crear columnas independientes de ceros y unos.

<div class="alert alert-block alert-info">
    <b>Ejercicio:</b> Utiliza el método <code>pd.get_dummies()</code> de <code>pandas</code> pasando <code>data_teams, dtype=int</code> como parámetros, ¿Qué sucede?.
    <hr>
    Cuando lo entiendas, sobreescribe <code>data_teams</code>.
</div>

In [ ]:
# Pruebas

In [ ]:
Y_sk = data_teams["Team"] # Guardamos la columna original para modelos sklearn
data_teams = # Tu código aquí

<div class="alert alert-block alert-info">
    <b>Ejercicio:</b> Separa la X e Y del dataframe <code>data_teams</code>, divide en entrenamiento y test (<code>Y_sk</code> incluida) (80/20) fijando la semilla y finalmente <b>normaliza</b> con la clase <code>MinMaxScaler()</code> las X.
</div>

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

x_cols = ['Sector1Time', 'Sector2Time', 'Sector3Time', 'SpeedI1', 'SpeedI2', 'SpeedFL', 'SpeedST', 'TyreLife', 'Compound_HARD', 'Compound_INTERMEDIATE', 'Compound_MEDIUM', 'Compound_SOFT', 'Compound_WET']
y_keras_cols = ['Team_Alfa Romeo', 'Team_AlphaTauri', 'Team_Alpine', 'Team_Aston Martin', 'Team_Ferrari', 'Team_Haas F1 Team', 'Team_McLaren', 'Team_Mercedes', 'Team_Red Bull Racing', 'Team_Williams']

# Tu código aquí

# X_train, X_test, Y_train, Y_test, Y_sk_train, Y_sk_test = ...

### **3.2. Aprendizaje automático**

Con los datos listos, entrenaremos y evaluaremos de nuevo los modelos de aprendizaje automático ya conocidos para poder compararlos con nuestro nuevo sistema.

<div class="alert alert-block alert-warning">
    <strong>Nota:</strong> La métrica <code>f1_score</code> se puede obtener de diferentes formas en problemas con múltiples clases:
    <ul>
        <li><strong>micro</strong>: Calcula la métrica global considerando todas las muestras, sin distinguir entre clases. Es útil cuando las clases están desbalanceadas.</li>
        <li><strong>macro</strong>: Calcula la métrica de cada clase por separado y luego hace el promedio aritmético. Da el mismo peso a todas las clases, sin importar su frecuencia.</li>
        <li><strong>weighted</strong>: Similar a macro, pero pondera cada clase según su número de muestras. Es útil cuando las clases están desbalanceadas.</li>
        <li><strong>samples</strong>: Se usa en problemas multietiqueta, calculando la métrica para cada muestra y luego promediando.</li>
    </ul>
</div>

Como ya has visto en el histograma anterior, las clases están balanceadas, por lo que podemos utilizar `macro`.

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier

# Baseline Random
baseline_random = DummyClassifier(strategy="uniform")
baseline_random.fit(X_train, Y_sk_train)
preds_test = baseline_random.predict(X_test)
evaluate_model(Y_sk_test, preds_test, "Baseline Random", average="macro")

# Baseline Zero-R
baseline_zero = DummyClassifier(strategy="most_frequent")
baseline_zero.fit(X_train, Y_sk_train)
preds_test = baseline_zero.predict(X_test)
evaluate_model(Y_sk_test, preds_test, "Baseline Zero-R", average="macro")

# KNN
model_knn = KNeighborsClassifier()
model_knn.fit(X_train, Y_sk_train)
preds_test = model_knn.predict(X_test)
evaluate_model(Y_sk_test, preds_test, "KNN", average="macro")

# Árboles de Decisión
model_tree = DecisionTreeClassifier()
model_tree.fit(X_train, Y_sk_train)
preds_test = model_tree.predict(X_test)
evaluate_model(Y_sk_test, preds_test, "Tree", average="macro")

Los resultados han de ser algo así:

<center>

| Modelo              | Accuracy (test) | F1 macro (test) |
|---------------------|-----------------|-----------------|
| Baseline Random     | 0.100           | 0.097           |
| Baseline Zero-R     | 0.073           | 0.013           |
| KNN                 | 0.565           | 0.567           |
| Árboles de Decisión | 0.521           | 0.519           |

</center>

<div class="alert alert-block alert-warning">
    <strong>Nota:</strong> Como verás, no estamos utilizando modelos como la <b>Regresión Logística</b> o las <b>Máquinas de vectores soporte</b>. Estos modelos <u>solo funcionan para resolver problemas de clasificación binaria</u>, aunque existen formas de adaptarlos a problemas multiclase. 
</div>

### **3.3. Aprendizaje profundo**

Una vez tenemos varios modelos de aprendizaje automático entrenados para resolver nuestro problema, intentaremos crear una *red neuronal* con el objetivo de mejorar los resultados.

Al ser un problema de clasificación, buscamos obtener valores entre $0$ y $1$ en la salida (probabilidades), por lo que podemos pensar que es necesario ubicar una `sigmoid` en la última capa.

El problema radica en que, en la clasificación multiclase, tenemos tantas salidas como clases, pero **solo una de ellas puede valer uno**, ya que cada ejemplo pertenece a una única clase.

<div class="alert alert-block alert-warning">
    <strong>Nota:</strong> Si utilizamos una función <code>sigmoid</code> en la capa final de un modelo con múltiples salidas, cada salida tendrá un valor entre cero y uno, lo que <b>no garantiza que <u>solo una</u> de las salidas tenga un valor de uno</b>.
</div>

Como ves, la `sigmoid` no es buena opción en este caso, es por eso en este tipo de problemas utilizaremos la `softmax`.

También será necesario utilizar una *función de pérdida* que tenga en cuenta este escenario multiclase. Esta es la llamada `Categorical Crossentropy`.

Actualizando nuestra tabla de *cambios* en redes neuronales según el problema, obtenemos lo siguiente:

<center>

| Tipo de problema              | Función de activación en la última capa         | Función de pérdida         | En *keras*                                  |
|-------------------------------|-------------------------------------------------|----------------------------|---------------------------------------------|
| *Regresión*                   | Ninguna o *ReLU* (si los valores son positivos) | *MAE* o *MSE*              | `mean_average_error` o `mean_squared_error` |
| *Clasificación Binaria*       | *Sigmoide*                                      | *Binary Crossentropy*      | `binary_crossentropy`                       |
| *Clasificación Multiclase*    | *Softmax*                                       | *Categorical Crossentropy* | `categorical_crossentropy`                  |

</center>


<div class="alert alert-block alert-info">
    <b>Ejercicio:</b> Crea una red neuronal de multiclasificación no lineal para intentar mejorar los modelos de aprendizaje automático tradicionales en esta tarea. Busca el mejor <code>learning rate</code>, entrena y evalúa en test el modelo final. Rellena ambas tablas.
    <hr style="margin-bottom:5px">
    Fija el conjunto de validación al 20%, las epocas a 200 y el batch a 64. Recuerda que para entrenar el modelo final no es necesario el conjunto de validación.
</div>


<center>

| Modelo                     | Loss (train)  | Loss (val) |
|----------------------------|---------------|------------|
| *Red Neuronal (lr=0.001)*  |               |            |
| *Red Neuronal (lr=0.005)*  |               |            |
| *Red Neuronal (lr=0.01)*   |               |            |

</center>

<br>

<center>

| Modelo              | Accuracy (test) | F1 macro (test) |
|---------------------|-----------------|-----------------|
| Baseline Random     | 0.100           | 0.097           |
| Baseline Zero-R     | 0.073           | 0.013           |
| KNN                 | 0.565           | 0.567           |
| Árboles de Decisión | 0.521           | 0.519           |
| Red Neuronal        |                 |                 |

</center>

In [ ]:
from tensorflow.keras.metrics import F1Score

def red_neuronal_multiclass(learning_rate):
    # Creamos y compilamos el modelo
    
    # Tu código aquí

    f1 = F1Score(average="macro")
    model.compile(loss= "", # Completar
                  optimizer=optim, 
                  metrics=[f1])

    return model

# Creamos la red desde cero
model_mtc = red_neuronal_multiclass(learning_rate = 0.001)

# Entrenamos
# Tu código aquí

# Visualizamos el entrenamiento
# Tu código aquí

In [ ]:
# Entrenar modelo final (sin validación)
# Tu código aquí

# Evaluar en test
# Tu código aquí

<hr>

## **4. Problemas multietiqueta**

El último tipo de problema que nos queda por ver es la clasificación **multietiqueta**, es decir, un problema donde cada ejemplo puede pertenecer **a una o varias clases**.

<div class="alert alert-block alert-success">
    <b>Desarrolla un modelo que, dado el tipo de neumático y la velocidad en el primer sector (<code>"Compound" y "SpeedI1"</code>), pueda predecir el <i>piloto o pilotos</i> (<code>"Driver"</code>) que han utilizado dicha combinación.</b>
</div>
  
Como siempre, lo primero será crear el dataset necesario para entrenar y evaluar los diferentes modelos.

### **4.1. Preprocesado de datos**

Creamos la variable <code>data_drivers</code> con las filas y columnas necesarias para entrenar nuestros modelos.

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

data_drivers = data.groupby(["Compound","SpeedI1"])["Driver"].apply(lambda x: x.unique()).reset_index()
# Se codifican los pilotos como multi-hot
mlb = MultiLabelBinarizer()
driver_dummies = pd.DataFrame(mlb.fit_transform(data_drivers["Driver"]), columns=map(lambda x: "Driver_"+str(x),mlb.classes_))
# Se añaden las nuevas columnas codificadas como números
data_drivers = data_drivers = pd.concat([data_drivers.drop(columns=["Driver"]), driver_dummies], axis=1)

<div class="alert alert-block alert-info">
    <b>Ejercicio:</b> Separa las X e Y del dataframe <code>data_drivers</code>, divide en entrenamiento y test (80/20) fijando la semilla y finalmente <b>normaliza</b> con la clase <code>MinMaxScaler()</code> las X.
    <hr style="margin-bottom:5px">
    Puede que tengas que codificar alguna de las columnas.
</div>

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Tu código aquí

### **4.2. Aprendizaje automático**

Con los datos listos, entrenaremos y evaluaremos los modelos de aprendizaje automático ya conocidos para poder compararlos con nuestro nuevo sistema.

<div class="alert alert-block alert-warning">
    <strong>Nota:</strong> Recuerda que la métrica <code>f1_score</code> se puede obtener de diferentes formas en problemas con múltiples clases. En este caso <code>samples</code> parece la mejor opción.
</div>


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier

# Baseline Random
baseline_random = DummyClassifier(strategy="uniform")
baseline_random.fit(X_train, Y_train)
preds_test = baseline_random.predict(X_test)
evaluate_model(Y_test, preds_test, "Baseline Random", average="samples")

# Baseline Zero-R
baseline_zero = DummyClassifier(strategy="most_frequent")
baseline_zero.fit(X_train, Y_train)
preds_test = baseline_zero.predict(X_test)
evaluate_model(Y_test, preds_test, "Baseline Zero-R", average="samples")

# KNN
model_knn = KNeighborsClassifier()
model_knn.fit(X_train, Y_train)
preds_test = model_knn.predict(X_test)
evaluate_model(Y_test, preds_test, "KNN", average="samples")

# Árboles de Decisión
model_tree = DecisionTreeClassifier()
model_tree.fit(X_train, Y_train)
preds_test = model_tree.predict(X_test)
evaluate_model(Y_test, preds_test, "Tree", average="samples")

Los resultados han de ser algo así:

<center>

| Modelo              | Accuracy (test) | F1 samples (test) |
|---------------------|-----------------|-------------------|
| Baseline Random     | 0.000           | 0.206             | 
| Baseline Zero-R     | 0.000           | 0.000             |
| KNN                 | 0.143           | 0.452             |
| Árboles de Decisión | 0.190           | 0.487             |

</center>


### **4.3. Aprendizaje profundo**

Como siempre, una vez tenemos varios modelos de aprendizaje automático entrenados para resolver nuestro problema, intentaremos crear una *red neuronal* con el objetivo de mejorar los resultados.

Como recordarás, en los problemas multiclase teníamos varias salidas (tantas como clases) y cada ejemplo solo podía pertenecer a una clase. En los problemas de **clasificación multietiqueta** como este, también tenemos tantas salidas como clases pero ahora <u>un ejemplo puede pertenecer a una o varias clases</u>. 

En lo que respecta a nuestra red neuronal, esto implica que podremos tener varios unos a la salida, por tanto podemos utilizar una `sigmoid`. 

<div class="alert alert-block alert-warning">
    <strong>Nota:</strong> Los problemas de clasificación multietiqueta se pueden ver como <i>múltiples problemas de clasificación binaria en paralelo</i>.
</div>

Utilizaremos por tanto la misma loss y función de activación en la última capa que en clasificación binaria. Actualizando nuestra tabla de *cambios* en redes neuronales según el problema, obtenemos lo siguiente:

<center>

| Tipo de problema              | Función de activación en la última capa         | Función de pérdida         | En *keras*                                  |
|-------------------------------|-------------------------------------------------|----------------------------|---------------------------------------------|
| *Regresión*                   | Ninguna o *ReLU* (si los valores son positivos) | *MAE* o *MSE*              | `mean_average_error` o `mean_squared_error` |
| *Clasificación Binaria*       | *Sigmoide*                                      | *Binary Crossentropy*      | `binary_crossentropy`                       |
| *Clasificación Multiclase*    | *Softmax*                                       | *Categorical Crossentropy* | `categorical_crossentropy`                  |
| *Clasificación Multietiqueta* | *Sigmoide*                                      | *Binary Crossentropy*      | `binary_crossentropy`                       |

</center>

<div class="alert alert-block alert-info">
    <b>Ejercicio:</b> Crea una red neuronal multietiqueta no lineal para intentar mejorar los modelos de aprendizaje automático tradicionales en esta tarea. Busca el mejor <code>learning rate</code>, entrena y evalúa en test el modelo final. Rellena ambas tablas.
    <hr style="margin-bottom:5px">
    Fija el conjunto de validación al 20%, las epocas a 200 y el batch a 64. Recuerda que para entrenar el modelo final no es necesario el conjunto de validación.
</div>

<center>

| Modelo                     | Loss (train)  | Loss (val) |
|----------------------------|---------------|------------|
| *Red Neuronal (lr=0.001)*  |               |            |
| *Red Neuronal (lr=0.005)*  |               |            |
| *Red Neuronal (lr=0.01)*   |               |            |

</center>
<br>
<center>

| Modelo              | Accuracy (test) | F1 samples (test) |
|---------------------|-----------------|-------------------|
| Baseline Random     | 0.000           | 0.206             | 
| Baseline Zero-R     | 0.000           | 0.000             |
| KNN                 | 0.143           | 0.452             |
| Árboles de Decisión | 0.190           | 0.487             |
| Red Neuronal        |                 |                   |

</center>


In [ ]:
from tensorflow.keras.metrics import F1Score

def red_neuronal_multilabel(learning_rate):
    # Creamos y compilamos el modelo
    
    # Tu código aquí
    f1 = F1Score(average="macro")

    return model

# Creamos la red desde cero
model_mtl = red_neuronal_multilabel(learning_rate = 0.001)

# Entrenamos
# Tu código aquí

# Visualizamos el entrenamiento
# Tu código aquí

In [ ]:
# Entrenar modelo final (sin validación)
# Tu código aquí

# Evaluar en test
# Tu código aquí

<hr>

## **5. Ejercicios**


<div class="alert alert-block alert-success">
    <b>Crear un modelo que, a partir del tiempo (en segundos) de los sectores (<code>"Sector1Time", "Sector2Time" y "Sector3Time"</code>) y las velocidades (<code>"SpeedI1", "SpeedI2", "SpeedFL" y "SpeedST"</code>), pueda predecir el <i>neumático</i> (<code>"Compound"</code>) utilizado en la vuelta.</b>
</div>

In [ ]:
# Tu código aquí